<a href="https://colab.research.google.com/github/Kingfinalweapon/kings/blob/master/%E3%80%90%E7%95%AA%E7%B5%84%E8%A1%A8%E3%83%BB%E7%9B%B4%E5%89%8D%E6%83%85%E5%A0%B1%E3%80%91%E3%83%9C%E3%83%BC%E3%83%88%E3%83%AC%E3%83%BC%E3%82%B9%E7%9B%B4%E5%89%8D%E6%83%85%E5%A0%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime as dt
import pandas as pd
from tqdm.notebook import tqdm
import datetime
import time
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
!pip install requests
from bs4 import BeautifulSoup
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 直前情報
def last_information(url):
  # 展示タイム
  df_time = pd.read_html(url)[1]
  df_time.columns= ['枠','写真','ボートレーサー','体重','展示タイム','チルト','プロペラ','部品交換','前走成績','前走成績']
  df_time.drop_duplicates(subset='ボートレーサー',inplace=True)
  df_time.reset_index(drop=True,inplace=True)
  df_result = df_time[['展示タイム']]
  # ST
  df_st_before = pd.read_html(url)[2]
  df_st_before.columns = ['前コース','並び','前ST']
  df_st_before['コース'] = df_st_before['前コース'].map(lambda x: int(x[:1]))
  df_st_before['ST'] = df_st_before['前コース'].map(lambda x: x[1:].strip())
  df_st_before.drop(['前コース','並び','前ST'],inplace=True,axis=1)

  for i in range(len(df_st_before)):
    if re.compile('\.[0-9]{2}').match(df_st_before['ST'].iloc[i]):
      df_st_before['ST'].iloc[i] = df_st_before['ST'].iloc[i].zfill(4)
  df_st_before

  # weather	wind wave
  res = requests.get(url)
  soup = BeautifulSoup(res.text,'html.parser')
  weather = soup.select('.is-weather .weather1_bodyUnitLabelTitle')[0].string
  wind = soup.select('.is-wind .weather1_bodyUnitLabelData')[0].string
  wave = soup.select('.is-wave .weather1_bodyUnitLabelData')[0].string
  df_st_before['weather'] = weather
  df_st_before['wind'] = wind[:1]
  df_st_before['wave'] = wave[:1]

  # race_id
  date = url[-8:]
  rounde = url.split("=")[1].split("&")[0].zfill(2)
  place_code = url.split("=")[2].split("&")[0]
  rece_code =  date + place_code + rounde
  df_st_before['race_code'] = rece_code

  # wind_d
  wind_d_before = soup.select('.weather1_bodyUnit.is-windDirection')[0].find('p')
  wind_d_before = wind_d_before.get('class')[1]
  wind_d_before = int(re.findall('\d+',wind_d_before)[0])
  wind_d_list = {1:3,2:3, 3:4,4:4,  5:5,6:5,  7:6,8:6,  9:7,10:7,  11:8,12:8, 13:1,14:1, 15:2,16:2}
  wind_d = wind_d_list[wind_d_before]
  df_st_before['wind_d'] = wind_d
  df_st_before.sort_values('コース',inplace=True)
  df_st_before = df_st_before.reindex(columns = ['race_code','ST','weather','wave','wind','wind_d','コース'])
  df_st_before.reset_index(drop=True,inplace=True)
  return df_st_before

In [ ]:
# 番組表
def time_information(url):
  url_race = url.replace(".jp/owpc/pc/race/beforeinfo?", ".jp//owpc/pc/race/racelist?")
  # 出走表
  tab = pd.DataFrame()
  time.sleep(1)
  html = requests.get(url_race)
  html.encoding = "EUC-JP"
  soup = BeautifulSoup(html.content, "html.parser")
  date = url_race[-8:]
  race_code = date
  rounde = url_race.split("=")[1].split("&")[0].zfill(2)
  place_code = url_race.split("=")[2].split("&")[0]
  race_code =  date + place_code + rounde
  date = date[0:4] + "/" + date[4:6] + "/" + date[6:8]
  term = url_race.split("=")[1].split("&")[0] + "R"
  personals = soup.find_all("tbody", attrs={"class": "is-fs12"})
  touban = []
  kyubetsu = []
  shibu = []
  age = []
  weight = []
  zen_syou = []
  zen_2ren = []
  tou_syou = []
  tou_2ren = []
  mot_no = []
  mot_2ren = []
  boa_no = []
  boa_2ren = []
  hayami = []
  for personal in personals:
      infos = personal.find_all("div", attrs={"class":"is-fs11"})
      touban.append(re.findall("\w+", infos[0].text)[0])
      kyubetsu.append(re.findall("\w+", infos[0].text)[1])
      shibu.append(re.findall("\w+", infos[1].text)[0])
      age.append(re.findall("\w+", infos[1].text)[2][:-1])
      weight.append(".".join(re.findall("\w+", infos[1].text)[3:])[:-2])
      zen_syou.append(".".join(re.findall("\d+", personal.find_all("td", attrs={"class":"is-lineH2"})[1].text)[0:2]))
      zen_2ren.append(".".join(re.findall("\d+", personal.find_all("td", attrs={"class":"is-lineH2"})[1].text)[2:4]))
      tou_syou.append(".".join(re.findall("\d+", personal.find_all("td", attrs={"class":"is-lineH2"})[2].text)[0:2]))
      tou_2ren.append(".".join(re.findall("\d+", personal.find_all("td", attrs={"class":"is-lineH2"})[2].text)[2:4]))
      mot_no.append(re.findall("\d+", personal.find_all("td", attrs={"class":"is-lineH2"})[3].text)[0])
      mot_2ren.append(".".join(re.findall("\d+", personal.find_all("td", attrs={"class":"is-lineH2"})[3].text)[1:3]))
      boa_no.append(re.findall("\d+", personal.find_all("td", attrs={"class":"is-lineH2"})[4].text)[0])
      boa_2ren.append(".".join(re.findall("\d+", personal.find_all("td", attrs={"class":"is-lineH2"})[4].text)[1:3]))
      hayami.append(personal.find_all("td", attrs={"rowspan": "4"})[-1].text[:-1])
  for tei in range(0, 6):
      temp_df = pd.DataFrame([(race_code, tei+1, touban[tei], kyubetsu[tei], shibu[tei], age[tei], weight[tei], zen_syou[tei], zen_2ren[tei],
                              tou_syou[tei],tou_2ren[tei], mot_no[tei], mot_2ren[tei], boa_no[tei], boa_2ren[tei], hayami[tei])])\
                  .rename(columns={0:"race_code", 1:"艇番", 2:"登録番号", 3:"級別", 4:"支部", 5:"年齢", 6:"体重", 7:"全国勝率", 8:"全国2連対率",
                                  9:"当地勝率", 10:"当地2連対率", 11:"モーター番号", 12:"モーター2連対率", 13:"ボート番号",
                                  14:"ボート2連対率", 15:"早見"})
      tab = pd.concat([tab, temp_df])
  tab = tab.sort_values(["race_code", "艇番"])
  tab.reset_index(drop=True,inplace=True)
  return tab



In [ ]:
last_information = last_information('https://www.boatrace.jp/owpc/pc/race/beforeinfo?rno=3&jcd=01&hd=20230329')
time_information = time_information('https://www.boatrace.jp/owpc/pc/race/beforeinfo?rno=3&jcd=01&hd=20230329')

In [ ]:
merge_data = pd.merge(time_information,last_information,left_on=['race_code','艇番'],right_on=['race_code','コース'])
del merge_data['コース']
merge_data

,race_code,艇番,登録番号,級別,支部,年齢,体重,全国勝率,全国2連対率,当地勝率,...,モーター番号,モーター2連対率,ボート番号,ボート2連対率,早見,ST,weather,wave,wind,wind_d
0,202303290103,1,4563,B1,岡山,35,52.4,4.06,18.60,0.00,...,74,19.57,64,47.69,9,F.08,晴,1,2,7
1,202303290103,2,4411,A2,大阪,37,53.2,5.50,36.67,4.80,...,48,26.83,19,17.74,11,0.11,晴,1,2,7
2,202303290103,3,4527,B1,愛知,34,51.5,4.09,21.84,3.90,...,30,27.91,72,37.50,7,0.15,晴,1,2,7
3,202303290103,4,4169,B1,東京,40,52.7,4.04,23.53,4.03,...,14,37.21,24,29.85,8,0.04,晴,1,2,7
4,202303290103,5,5279,B2,東京,20,52.0,2.75,6.25,2.83,...,41,31.71,51,46.15,8,0.15,晴,1,2,7
5,202303290103,6,3687,B1,愛知,53,53.5,4.34,21.05,5.30,...,66,32.61,13,30.51,,F.06,晴,1,2,7
